In [2]:
PollTrainImp = read.csv("pollTrain_imputed.csv")

In [3]:
str(PollTrainImp)

'data.frame':	5568 obs. of  108 variables:
 $ USER_ID        : int  1 4 5 8 9 10 11 12 13 15 ...
 $ YOB            : int  1938 1970 1997 1983 1984 1997 1983 1996 1994 1981 ...
 $ Gender         : Factor w/ 2 levels "Female","Male": 2 1 2 2 1 1 2 2 2 1 ...
 $ Income         : Factor w/ 6 levels "$100,001 - $150,000",..: 4 5 4 1 3 5 2 4 6 3 ...
 $ HouseholdStatus: Factor w/ 6 levels "Domestic Partners (no kids)",..: 4 2 5 4 4 5 3 5 5 4 ...
 $ EducationLevel : Factor w/ 7 levels "Associate's Degree",..: 2 2 6 2 6 3 4 3 3 2 ...
 $ Party          : Factor w/ 2 levels "Democrat","Republican": 1 1 2 1 2 1 1 2 2 2 ...
 $ Q124742        : Factor w/ 2 levels "No","Yes": 1 1 1 1 1 1 1 2 1 1 ...
 $ Q124122        : Factor w/ 2 levels "No","Yes": 1 2 2 2 2 1 1 2 1 1 ...
 $ Q123464        : Factor w/ 2 levels "No","Yes": 1 1 2 1 1 1 1 1 2 1 ...
 $ Q123621        : Factor w/ 2 levels "No","Yes": 1 1 1 2 1 1 2 1 1 1 ...
 $ Q122769        : Factor w/ 2 levels "No","Yes": 1 1 2 1 1 1 2 1 2 2 ...
 $ Q122

In [4]:
library(caTools)

In [5]:
t = PollTrainImp$Party
t = ifelse(PollTrainImp$Party=="Democrat", 1, 0)

In [6]:
PollTrainImp$Party = t

In [10]:
table(PollTrainImp$Party)


   0    1 
2617 2951 

In [12]:
PollTrainImp$Party = as.factor(PollTrainImp$Party)

In [13]:
table(PollTrainImp$Party)


   0    1 
2617 2951 

In [14]:
library(caret)

Loading required package: lattice
Loading required package: ggplot2
Warning message:
: package ‘ggplot2’ was built under R version 3.3.0

In [15]:
library(e1071)

In [16]:
numFolds = trainControl(method = "cv",number = 10)

In [17]:
cpGrid = expand.grid(.cp=seq(0.01,0.5,0.01))

In [19]:
summary(cpGrid)

      .cp        
 Min.   :0.0100  
 1st Qu.:0.1325  
 Median :0.2550  
 Mean   :0.2550  
 3rd Qu.:0.3775  
 Max.   :0.5000  

In [20]:
train(Party~ . - USER_ID,data = PollTrainImp, method = "rpart", trControl = numFolds, tuneGrid = cpGrid)

Loading required package: rpart


CART 

5568 samples
 107 predictors
   2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 5011, 5011, 5012, 5011, 5010, 5011, ... 
Resampling results across tuning parameters:

  cp    Accuracy   Kappa       Accuracy SD   Kappa SD  
  0.01  0.6233794  0.24392217  0.0240251402  0.04833846
  0.02  0.6233794  0.24392217  0.0240251402  0.04833846
  0.03  0.6073919  0.21900077  0.0211693359  0.03666324
  0.04  0.5937442  0.20795853  0.0169929450  0.03281077
  0.05  0.5937442  0.20795853  0.0169929450  0.03281077
  0.06  0.5937442  0.20795853  0.0169929450  0.03281077
  0.07  0.5937442  0.20795853  0.0169929450  0.03281077
  0.08  0.5937442  0.20795853  0.0169929450  0.03281077
  0.09  0.5937442  0.20795853  0.0169929450  0.03281077
  0.10  0.5937442  0.20795853  0.0169929450  0.03281077
  0.11  0.5937442  0.20795853  0.0169929450  0.03281077
  0.12  0.5937442  0.20795853  0.0169929450  0.03281077
  0.13  0.5836903  0.18027249  0.022043742

In [21]:
CARTModel1 = rpart(Party~. - USER_ID, data = PollTrainImp, method = "class", cp = 0.02)

In [22]:
PollTestImp = read.csv("pollTest_imputed.csv")

In [23]:
Predictions = predict(CARTModel1, newdata = PollTestImp, type = "class")

In [24]:
table(Predictions)

Predictions
  0   1 
666 726 

In [25]:
PredTestLabels = as.factor(ifelse(Predictions == 1, "Democrat", "Republican"))

In [26]:
table(PredTestLabels)

PredTestLabels
  Democrat Republican 
       726        666 

In [27]:
MySubmission = data.frame(USER_ID = PollTestImp$USER_ID, Predictions = PredTestLabels)
str(MySubmission)
write.csv(MySubmission, "SubmissionCART_CV.csv", row.names=FALSE)

'data.frame':	1392 obs. of  2 variables:
 $ USER_ID    : int  2 3 6 7 14 28 29 37 44 56 ...
 $ Predictions: Factor w/ 2 levels "Democrat","Republican": 1 2 2 1 1 2 1 2 1 1 ...


In [28]:
predTrain = predict(CARTModel1, newdata = PollTrainImp, type = "class")

In [29]:
t= table(PollTrainImp$Party,predTrain)
t

   predTrain
       0    1
  0 1565 1052
  1 1045 1906

In [30]:
sum(diag(t))/nrow(PollTrainImp)

[1] 0.6233836

In [31]:
pollTrain = read.csv("train2016.csv")

In [32]:
str(pollTrain)

'data.frame':	5568 obs. of  108 variables:
 $ USER_ID        : int  1 4 5 8 9 10 11 12 13 15 ...
 $ YOB            : int  1938 1970 1997 1983 1984 1997 1983 1996 NA 1981 ...
 $ Gender         : Factor w/ 3 levels "","Female","Male": 3 2 3 3 2 2 3 3 3 2 ...
 $ Income         : Factor w/ 7 levels "","$100,001 - $150,000",..: 1 6 5 2 4 6 3 5 1 4 ...
 $ HouseholdStatus: Factor w/ 7 levels "","Domestic Partners (no kids)",..: 5 3 6 5 5 6 4 6 6 5 ...
 $ EducationLevel : Factor w/ 8 levels "","Associate's Degree",..: 1 3 7 3 7 4 5 4 4 1 ...
 $ Party          : Factor w/ 2 levels "Democrat","Republican": 1 1 2 1 2 1 1 2 2 2 ...
 $ Q124742        : Factor w/ 3 levels "","No","Yes": 2 1 1 2 2 1 1 3 2 2 ...
 $ Q124122        : Factor w/ 3 levels "","No","Yes": 1 3 3 3 3 1 1 3 1 2 ...
 $ Q123464        : Factor w/ 3 levels "","No","Yes": 2 2 3 2 2 1 1 2 3 2 ...
 $ Q123621        : Factor w/ 3 levels "","No","Yes": 2 2 2 3 2 1 1 2 2 2 ...
 $ Q122769        : Factor w/ 3 levels "","No","Yes": 2 2 1 

In [33]:
pollTrain$Party = ifelse(pollTrain$Party=="Democrat", 1, 0)

In [34]:
table(pollTrain$Party)


   0    1 
2617 2951 

In [36]:
split = sample.split(pollTrain$Party, SplitRatio = 0.7)

In [37]:
Train = subset(pollTrain,split == TRUE)
Test = subset(pollTrain, split == FALSE)

In [38]:
numFolds = trainControl(method = "cv",number = 10)
cpGrid = expand.grid(.cp=seq(0.01,0.5,0.01))
summary(cpGrid)

      .cp        
 Min.   :0.0100  
 1st Qu.:0.1325  
 Median :0.2550  
 Mean   :0.2550  
 3rd Qu.:0.3775  
 Max.   :0.5000  

In [40]:
pollTrain$Party = as.factor(pollTrain$Party)

In [41]:
train(Party~ . - USER_ID,data = pollTrain, method = "rpart", trControl = numFolds, tuneGrid = cpGrid)

CART 

5568 samples
 107 predictors
   2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 4712, 4711, 4711, 4711, 4711, 4712, ... 
Resampling results across tuning parameters:

  cp    Accuracy   Kappa      Accuracy SD   Kappa SD  
  0.01  0.6189139  0.2227836  0.0223783108  0.04513673
  0.02  0.6189139  0.2227836  0.0223783108  0.04513673
  0.03  0.6189139  0.2227836  0.0223783108  0.04513673
  0.04  0.6189139  0.2227836  0.0223783108  0.04513673
  0.05  0.5988608  0.1742822  0.0213379064  0.04835300
  0.06  0.5725019  0.1716044  0.0135900049  0.03639464
  0.07  0.5711661  0.1755347  0.0151924165  0.02858508
  0.08  0.5711661  0.1755347  0.0151924165  0.02858508
  0.09  0.5339031  0.0262520  0.0073656726  0.05537397
  0.10  0.5304679  0.0000000  0.0007016811  0.00000000
  0.11  0.5304679  0.0000000  0.0007016811  0.00000000
  0.12  0.5304679  0.0000000  0.0007016811  0.00000000
  0.13  0.5304679  0.0000000  0.0007016811  0.00000000


In [43]:
CARTModel2 = rpart(Party~. -USER_ID, data = pollTrain, method = "class", cp = 0.04)

In [44]:
pollTest = read.csv("test2016.csv")

In [45]:
PredCartTest = predict(CARTModel2, newdata = pollTest, type = "class")

In [46]:
table(PredCartTest)

PredCartTest
  0   1 
487 905 

In [48]:
PredTestLabels2 = as.factor(ifelse(PredCartTest==1, "Democrat", "Republican"))
table(PredTestLabels2)

# However, you can submit the file on Kaggle to see how well the model performs. You can make up to 5 submissions per day, so don't hesitate to just upload a solution to see how you did.

# Let's prepare a submission file for Kaggle (for more about this, see the "Evaluation" page on the competition site):




PredTestLabels2
  Democrat Republican 
       905        487 

ERROR: Error in data.frame(USER_ID = pollTestt$USER_ID, Predictions = PredTestLabels2): object 'pollTestt' not found


In [49]:
MySubmission = data.frame(USER_ID = pollTest$USER_ID, Predictions = PredTestLabels2)

write.csv(MySubmission, "SubmissionRPART_NO_IMP_CV.csv", row.names=FALSE)